In [18]:
import os

path = "."
file_list = os.listdir(path)



In [19]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.multioutput import MultiOutputRegressor
import optuna
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
from sklearn.metrics import mean_absolute_error
import shap

from lightgbm import LGBMRegressor, LGBMClassifier

from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
import warnings ; warnings.filterwarnings('ignore')
import time
from sklearn.metrics import f1_score, roc_auc_score, classification_report


train=pd.read_csv(path + '/train.csv', index_col='id')
test=pd.read_csv(path + '/test.csv', index_col='id')
submission=pd.read_csv(path + '/sample_submission.csv', index_col='id')

print(train.shape, test.shape, submission.shape)



(10000, 75) (10000, 71) (10000, 4)


In [29]:
def datafill(data):
    data_dst = data.filter(regex='_dst$', axis=1).replace(0, np.NaN)
    data_dst = data_dst.interpolate(methods='linear',axis=1)
    data_dst.fillna(0,inplace=True)
    return data_dst

In [30]:
train = datafill(train)

In [31]:
train.head()

,650_dst,660_dst,670_dst,680_dst,690_dst,700_dst,710_dst,720_dst,730_dst,740_dst,...,900_dst,910_dst,920_dst,930_dst,940_dst,950_dst,960_dst,970_dst,980_dst,990_dst
id,,,,,,,,,,,,,,,,,,,,,
0,2.447758e-18,1.894578e-18,4.153745e-18,6.412913e-18,1.098559e-17,8.907142e-19,1.892920e-18,2.895126e-18,3.897332e-18,4.899539e-18,...,3.076794e-18,3.527371e-18,2.086464e-18,6.455564e-19,7.510433e-19,8.565303e-19,9.620173e-19,1.067504e-18,5.998949e-18,4.378513e-17
1,0.000000e+00,0.000000e+00,0.000000e+00,1.995530e-10,1.025259e-10,1.046220e-10,1.067180e-10,1.088140e-10,1.109100e-10,1.130060e-10,...,2.647633e-09,3.940557e-09,5.233480e-09,1.264238e-08,1.343132e-08,6.112685e-09,2.130547e-09,5.920319e-09,9.710091e-09,9.710091e-09
2,1.677445e-19,3.763590e-20,2.097417e-19,3.818476e-19,5.539534e-19,7.260593e-19,8.981651e-19,1.944683e-19,8.126907e-19,2.183200e-19,...,4.629125e-18,1.409413e-18,3.237480e-18,2.919521e-18,2.601562e-18,2.283602e-18,1.965643e-18,1.647684e-18,1.329725e-18,1.329725e-18
3,4.044975e-11,4.695169e-11,5.345363e-11,9.106886e-12,1.881458e-11,3.617201e-11,7.909656e-13,7.958742e-11,1.583839e-10,1.630944e-10,...,1.274876e-10,6.118947e-11,1.137891e-10,1.663888e-10,2.245998e-10,1.299511e-10,7.782625e-11,2.433592e-10,4.088921e-10,4.088921e-10
4,4.784751e-14,7.033465e-14,9.282179e-14,1.153089e-13,1.121056e-13,1.089024e-13,1.056991e-13,7.935493e-14,5.301078e-14,2.666663e-14,...,8.737755e-14,5.137687e-14,1.537619e-14,4.996522e-14,1.457955e-13,8.769053e-14,1.103571e-13,1.330237e-13,1.330237e-13,1.330237e-13
